In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Dense, GlobalAveragePooling2D, Flatten, BatchNormalization,Dropout
from keras.models import Model, model_from_json
import os
import matplotlib.pyplot as plt
import glob
import numpy as np

Using TensorFlow backend.


In [2]:
# dimensions of our images.
img_width, img_height = 64, 64

#Location of the images
train_data_dir = '../train'
val_data_dir = '../validation'
nb_epochs = 64
batch_size = 8
FC_SIZE = 256
NB_IV3_LAYERS_TO_FREEZE = 172

input_shape = (img_width, img_height, 3)

In [3]:
def get_no_files(dir):
    if not os.path.exists(dir):
        print("Error: Invalid directory")
        #TO DO: Error Handling
        return 0
    count = 0
    for r, dirs, files in os.walk(dir):
        for dr in dirs:
            count += len(glob.glob(os.path.join(r, dr + "/*")))
    return count

In [4]:
def freeze_all_layers(base_model):
    for layer in base_model.layers:
        layer.trainable=False
    

In [5]:
def add_new_last_layer(base_model, nb_classes):
    x = base_model.output
    #x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Flatten()(x)
    x = Dense(FC_SIZE, activation='elu')(x) #new FC layer, random init
    x = Dense(FC_SIZE, activation='elu')(x)
    x = BatchNormalization()(x)
    predictions = Dense(nb_classes, activation='softmax')(x) #new softmax layer
    model = Model(inputs=base_model.input, outputs=predictions)
    return model    

In [6]:

nb_train_samples = get_no_files(train_data_dir)
nb_classes = len(glob.glob(train_data_dir + "/*"))
nb_val_samples = get_no_files(val_data_dir)
nb_epochs = int(nb_epochs)
batch_size = int(batch_size)

# data prep
train_datagen =  ImageDataGenerator()
val_datagen = ImageDataGenerator(
    """preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True"""
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
)

validation_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
)



Found 4520 images belonging to 12 classes.
Found 240 images belonging to 12 classes.


In [7]:
# setup model
base_model = VGG16(weights='imagenet', input_shape=input_shape, include_top=False) #include_top=False excludes final FC layer
freeze_all_layers(base_model)
model = add_new_last_layer(base_model, nb_classes)

# transfer learning

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

"""history_tl = model.fit_generator(
    train_generator,
    nb_epoch=nb_epochs,
    steps_per_epoch=None,
    validation_data=validation_generator,
    nb_val_samples=nb_val_samples,
    class_weight='auto') 
"""
history_tl = model.fit_generator( 
                                 train_generator, 
                                 steps_per_epoch=int(nb_train_samples/batch_size), 
                                 epochs=nb_epochs, 
                                 validation_data=validation_generator, 
                                 validation_steps=int(nb_val_samples/batch_size))

Epoch 1/64
564/565 [============================>.] - ETA: 0s - loss: 2.0060 - acc: 0.3415

/home/pragya/.local/lib/python2.7/site-packages/keras/preprocessing/image.py:506: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


565/565 [==============================] - 500s - loss: 2.0048 - acc: 0.3418 - val_loss: 1.7104 - val_acc: 0.4458
Epoch 2/64
565/565 [==============================] - 502s - loss: 1.5654 - acc: 0.4650 - val_loss: 1.4863 - val_acc: 0.4542
Epoch 3/64
565/565 [==============================] - 501s - loss: 1.4395 - acc: 0.5159 - val_loss: 1.4401 - val_acc: 0.4292
Epoch 4/64
565/565 [==============================] - 501s - loss: 1.3376 - acc: 0.5489 - val_loss: 1.3239 - val_acc: 0.5292
Epoch 5/64
565/565 [==============================] - 502s - loss: 1.2731 - acc: 0.5688 - val_loss: 1.3116 - val_acc: 0.5917
Epoch 6/64
565/565 [==============================] - 502s - loss: 1.2256 - acc: 0.5770 - val_loss: 1.4546 - val_acc: 0.5083
Epoch 7/64
565/565 [==============================] - 502s - loss: 1.1474 - acc: 0.6031 - val_loss: 1.3655 - val_acc: 0.5333
Epoch 8/64
565/565 [==============================] - 508s - loss: 1.1220 - acc: 0.6250 - val_loss: 1.1571 - val_acc: 0.5500
Epoch 9/64


KeyboardInterrupt: 

In [8]:
#Save the model

# serialize model to JSON
model_json = model.to_json()
with open("plantSeed.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("plantSeed.h5")
print("Saved model to disk")

Saved model to disk


In [2]:
# load json and create model
json_file = open('plantSeed.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("plantSeed.h5")
print("Loaded model from disk")

Loaded model from disk


In [4]:
h=history_tl

plt.figure(0)
plt.plot(h.history['acc'],'r')
plt.plot(h.history['val_acc'],'g')
plt.xticks(np.arange(0, 80, 10.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("Accuracy")
plt.title("Training Accuracy vs Validation Accuracy")
plt.legend(['train','validation'])
 
 
plt.figure(1)
plt.plot(h.history['loss'],'r')
plt.plot(h.history['val_loss'],'g')
plt.xticks(np.arange(0, 80, 10.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("Loss")
plt.title("Training Loss vs Validation Loss")
plt.legend(['train','validation'])
 
 
plt.show()

NameError: name 'history_tl' is not defined